<a href="https://colab.research.google.com/github/MissTife/sync-internship/blob/main/chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install nltk
!pip install tensorflow

In [35]:
import json
import random
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
import numpy as np


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
import pickle
import tensorflow as tf
from  tensorflow.keras.models import Sequential
from  tensorflow.keras.layers import Dense, Dropout
from  tensorflow.keras.optimizers import SGD

In [44]:

def load_intents_from_json(file_path):
    with open("skincare.json",encoding='utf-8') as file:
        intents_data = json.load(file)
    return intents_data
intents_data = load_intents_from_json("skincare.json")


In [45]:
lemmatizer=WordNetLemmatizer()
words, classes,documents =[],[],[]
characters=['?', '!', '.',',']

In [46]:
for intent in intents_data['intents']:
    for pattern in intent['patterns']:
        words.extend(nltk.word_tokenize(pattern.lower()))
        documents.append((nltk.word_tokenize(pattern.lower()), intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [47]:
word= sorted(set([lemmatizer.lemmatize(word) for word in words if word not in characters]))
classes= sorted(set(classes))

In [48]:
train_data, output=[], [0] * len(classes)

for doc in documents:
    bag = [1 if lemmatizer.lemmatize(word) in doc[0] else 0 for word in words]
    output_row = list(output)
    output_row[classes.index(doc[1])] = 1
    train_data.append([bag, output_row])

random.shuffle(train_data)
x_train, y_train = np.array([data[0] for data in train_data]), np.array([data[1] for data in train_data])


In [49]:
model = Sequential([
    Dense(128, input_shape=(len(x_train[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(y_train[0]), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=200, batch_size=5, verbose=1)


model.save('my_chatbot')
with open('words.pkl', 'wb') as file:
    pickle.dump(words, file)
with open('classes.pkl', 'wb') as file:
    pickle.dump(classes, file)


Epoch 1/200
7/7 [==============================] - 1s 6ms/step - loss: 1.8629 - accuracy: 0.1613
Epoch 2/200
7/7 [==============================] - 0s 4ms/step - loss: 1.9216 - accuracy: 0.1613
Epoch 3/200
7/7 [==============================] - 0s 3ms/step - loss: 1.7704 - accuracy: 0.2903
Epoch 4/200
7/7 [==============================] - 0s 4ms/step - loss: 1.8436 - accuracy: 0.3226
Epoch 5/200
7/7 [==============================] - 0s 3ms/step - loss: 1.6958 - accuracy: 0.2903
Epoch 6/200
7/7 [==============================] - 0s 3ms/step - loss: 1.5925 - accuracy: 0.3548
Epoch 7/200
7/7 [==============================] - 0s 3ms/step - loss: 1.5951 - accuracy: 0.3871
Epoch 8/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4814 - accuracy: 0.4194
Epoch 9/200
7/7 [==============================] - 0s 3ms/step - loss: 1.4787 - accuracy: 0.4516
Epoch 10/200
7/7 [==============================] - 0s 4ms/step - loss: 1.4189 - accuracy: 0.5161
Epoch 11/200
7/7 [===========

In [50]:



def extract_names(input_text):
    words = input_text.split()
    if len(words) >= 1:
        name = " ".join(words)
        return name.strip()
    return None


def load_chatbot_model(model_path, words_path, classes_path):
    model = tf.keras.models.load_model(model_path)
    with open(words_path, 'rb') as file:
        words = pickle.load(file)
    with open(classes_path, 'rb') as file:
        classes = pickle.load(file)
    return model, words, classes

In [51]:
def get_chatbot_response(user_input, model, words, classes,intents_data):
    lemmatizer = WordNetLemmatizer()
    bag = [0] * len(words)
    user_words = nltk.word_tokenize(user_input.lower())
    user_words = [lemmatizer.lemmatize(word) for word in user_words]
    for w in user_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    results = model.predict(np.array([bag]))
    results_index = np.argmax(results)
    tag = classes[results_index]

    for intent in intents_data["intents"]:
        if intent["tag"] == tag:
            response = random.choice(intent["responses"])
            return response

    return "I am sorry, I do not understand. Please provide more information."

In [57]:
def chat():
    print("Chatbot: Welcome! What is your name?")
    name_requested = True


    # Load the trained model and word/class dictionaries
    model, words, classes = load_chatbot_model('my_chatbot', 'words.pkl', 'classes.pkl')

    while True:
        user_input = input("You: ").lower()

        if name_requested:
            name = extract_names(user_input)
            if name:
                print("Chatbot: Hello {name}! How can I help you today?".format(name=name))
                name_requested = False
            else:
                print("Chatbot: I am sorry, I do not have the information that you seek.")

        elif user_input == 'exit':
            print("ChatBot: Goodbye!")
            break
        else:
            response = get_chatbot_response(user_input, model, words, classes,intents_data)
            print("Chatbot:", response)

if __name__ == "__main__":
  print("We can chat now . Enter exit to end our chat")
  chat()








We can chat now . Enter exit to end our chat
Chatbot: Welcome! What is your name?
You: faith boluwatife
Chatbot: Hello faith boluwatife! How can I help you today?
You: how are you
1/1 [==============================] - 0s 109ms/step
Chatbot: I am good, thank you.
You: i have oily skin
1/1 [==============================] - 0s 27ms/step
Chatbot: Skincare for oily skin aims to control excess oil production, prevent breakouts, and maintain a balanced complexion. The goal is to keep the skin clean and well-moisturized without exacerbating oiliness. Here's a tailored skincare routine for oily skin:
1. Cleansing: Use a gentle, foaming cleanser specifically designed for oily or acne-prone skin. Cleansers with ingredients like salicylic acid or glycolic acid can help control oil and unclog pores.
2. Exfoliation: Exfoliate 2-3 times a week to remove dead skin cells and prevent clogged pores. Look for chemical exfoliants with ingredients like alpha hydroxy acids (AHAs) or beta hydroxy acids (BHA